In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [2]:
train_filename = "../data/samples/train_transactions_gt2018.tfrecord"
train = tf.data.TFRecordDataset(train_filename)

test_filename = "../data/samples/test_transactions_gt2018.tfrecord"
test = tf.data.TFRecordDataset(test_filename)

feature_description = {
    'context_item_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_item_quantity': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_item_price': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_item_department_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_discount_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_return_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),   
    'label_item_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
}

In [3]:
def _parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, feature_description)

train_ds = train.map(_parse_function).map(lambda x: {
    "context_item_id": tf.strings.as_string(x["context_item_id"]),
    "label_item_id": tf.strings.as_string(x["label_item_id"])
})

test_ds = test.map(_parse_function).map(lambda x: {
    "context_item_id": tf.strings.as_string(x["context_item_id"]),
    "label_item_id": tf.strings.as_string(x["label_item_id"])
})

In [4]:
for x in train_ds.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'context_item_id': array([b'95626', b'33553', b'33553', b'98609', b'150187', b'85219',
       b'85205', b'33553', b'91241', b'75745'], dtype=object),
 'label_item_id': array([b'75825'], dtype=object)}


In [5]:
items_filename = "../data/samples/items.tfrecord"
items_tf = tf.data.TFRecordDataset(items_filename)
item_feature_description = {
    'item_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
    'item_description': tf.io.FixedLenFeature([1], tf.string, default_value='None')}
def item_parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, item_feature_description)

items_ds = items_tf.map(item_parse_function).map(lambda x: {
    "item_id": tf.strings.as_string(x["item_id"]),
    "item_description": x["item_description"],
})
items = items_ds.map(lambda x: x["item_id"])
item_ids = items.batch(1_000)
unique_item_ids = np.unique(np.concatenate(list(item_ids)))

In [6]:
embedding_dimension = 32

query_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_item_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_item_ids) + 1, embedding_dimension), 
    tf.keras.layers.GRU(embedding_dimension),
])

candidate_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_item_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_item_ids) + 1, embedding_dimension)
])

In [7]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=items_tf.batch(128).map(candidate_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)

class MyModel(tfrs.Model):

    def __init__(self, query_model, candidate_model):
        super().__init__()
        self._query_model = query_model
        self._candidate_model = candidate_model

        self._task = task

    def compute_loss(self, features, training=False):
        purchase_history = features["context_item_id"]
        next_item = features["label_item_id"]

        query_embedding = self._query_model(purchase_history)       
        candidate_embedding = self._candidate_model(next_item)

        return self._task(query_embedding, candidate_embedding, compute_metrics=not training)

In [8]:
model = MyModel(query_model, candidate_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [9]:
cached_train = train_ds.shuffle(10_000).batch(12800).cache()
cached_test = test_ds.batch(2560).cache()

In [10]:
model.fit(cached_train, epochs=5)

Epoch 1/5
89/89 [==============================] - 63s 468ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 107068.8000 - regularization_loss: 0.0000e+00 - total_loss: 107068.8000
Epoch 2/5
89/89 [==============================] - 9s 104ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 98200.7333 - regularization_loss: 0.0000e+00 - total_loss: 98200.7333
Epoch 3/5
89/89 [==============================] - 9s 103ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 

In [11]:
model.evaluate(cached_test, return_dict=True)

110/110 [==============================] - 253s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.9034 - factorized_top_k/top_5_categorical_accuracy: 0.9034 - factorized_top_k/top_10_categorical_accuracy: 0.9034 - factorized_top_k/top_50_categorical_accuracy: 0.9034 - factorized_top_k/top_100_categorical_accuracy: 0.9034 - loss: 15468.9167 - regularization_loss: 0.0000e+00 - total_loss: 15468.9167


{'factorized_top_k/top_1_categorical_accuracy': 0.9034018516540527,
 'factorized_top_k/top_5_categorical_accuracy': 0.9034018516540527,
 'factorized_top_k/top_10_categorical_accuracy': 0.9034018516540527,
 'factorized_top_k/top_50_categorical_accuracy': 0.9034018516540527,
 'factorized_top_k/top_100_categorical_accuracy': 0.9034018516540527,
 'loss': 13.817108154296875,
 'regularization_loss': 0,
 'total_loss': 13.817108154296875}

In [12]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model._query_model, k=12)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((items.batch(100), items.batch(100).map(model._candidate_model)))
)

feature0 = ["7956" "112095" "90086" "75364" "48352" "144630" "85261" "75279" "128909" "128910"]
features_dataset = { 
  'context_item_id' : tf.constant(feature0,shape=(1,10,1)), 
}

_, predicted = index(features_dataset)
print(predicted)



array([[[b'79561120959008675364483521446308526175279128909128910'],
        [b'79561120959008675364483521446308526175279128909128910'],
        [b'79561120959008675364483521446308526175279128909128910'],
        [b'79561120959008675364483521446308526175279128909128910'],
        [b'79561120959008675364483521446308526175279128909128910'],
        [b'79561120959008675364483521446308526175279128909128910'],
        [b'79561120959008675364483521446308526175279128909128910'],
        [b'79561120959008675364483521446308526175279128909128910'],
        [b'79561120959008675364483521446308526175279128909128910'],
        [b'79561120959008675364483521446308526175279128909128910']]],
      dtype=object)>}. Consider rewriting this model with the Functional API.
tf.Tensor(
[[[b'92242']
  [b'12099']
  [b'92246']
  [b'70957']
  [b'92248']
  [b'138276']
  [b'92240']
  [b'70956']
  [b'70958']
  [b'89285']
  [b'130619']
  [b'92244']]], shape=(1, 12, 1), dtype=string)


In [13]:
path = '../data/simple_seq_model'
tf.saved_model.save(index, path)

INFO:tensorflow:Assets written to: ../data/simple_seq_model\assets


INFO:tensorflow:Assets written to: ../data/simple_seq_model\assets
